In [1]:
import numpy as np


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras import backend as K

num_cores = 6


num_CPU = 1
num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

In [3]:
X = np.random.random((1000, 4, 42, 42))
Y = np.random.random((1000, 6))

In [4]:
shared_layers = [
    Conv2D(16, kernel_size=8, strides=4, activation='relu', data_format='channels_first', input_shape=(4, 42, 42)),
    Conv2D(32, kernel_size=4, strides=2, activation='relu'),
    Flatten(),
    Dense(256, activation='relu')
]
models = [Sequential(shared_layers + [Dense(1)]) for _ in range(6)]

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
head = Sequential(shared_layers)

In [7]:
head.predict(X)

array([[0.01478189, 0.26165396, 0.1427239 , ..., 0.05160488, 0.00645338,
        0.        ],
       [0.        , 0.13743865, 0.25409406, ..., 0.02484468, 0.0188924 ,
        0.        ],
       [0.08294293, 0.07300003, 0.1463304 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09002445, 0.22982913, 0.223989  , ..., 0.02225919, 0.        ,
        0.        ],
       [0.09469388, 0.32019696, 0.00493235, ..., 0.12354706, 0.        ,
        0.        ],
       [0.08476409, 0.19119264, 0.17188776, ..., 0.03672082, 0.00828882,
        0.        ]], dtype=float32)

In [8]:
opt = Adam(lr=0.1,
           # beta_1=0.9, beta_2=0.999, epsilon=None,
           #decay=0.01,
           amsgrad=False)
for model in models:
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_error'])

In [11]:
from sklearn.metrics import mean_squared_error as mse

In [12]:
for i in range(6):
    print(f'i: {i}, mse: {mse(Y[:, i], models[i].predict(X))}')

i: 0, mse: 0.45428879678251
i: 1, mse: 0.19697493621726214
i: 2, mse: 0.5066699761717771
i: 3, mse: 0.34794254289492027
i: 4, mse: 0.16248628115963656
i: 5, mse: 0.3242728752341365


In [15]:
for i in range(6):
    print(f'i: {i}, mse: {mse(Y[:, i], models[i].predict(X))}')

i: 0, mse: 1114034578.677968
i: 1, mse: 2920932.8920064634
i: 2, mse: 641577.8275628586
i: 3, mse: 1622102.8846033793
i: 4, mse: 2220957.9595690393
i: 5, mse: 87983.12902133788


In [14]:
models[0].train_on_batch(X, Y[:, 0])

Instructions for updating:
Use tf.cast instead.


[0.4542888, 0.6060337]

In [16]:
head.predict(X)

array([[   0.    , 1418.4065,    0.    , ..., 1641.8231, 1559.7598,
           0.    ],
       [   0.    , 1430.6788,    0.    , ..., 1656.3696, 1573.008 ,
           0.    ],
       [   0.    , 1418.7236,    0.    , ..., 1641.8074, 1559.9995,
           0.    ],
       ...,
       [   0.    , 1422.3107,    0.    , ..., 1645.468 , 1563.926 ,
           0.    ],
       [   0.    , 1436.5985,    0.    , ..., 1662.3241, 1579.1593,
           0.    ],
       [   0.    , 1429.0363,    0.    , ..., 1653.8662, 1572.0508,
           0.    ]], dtype=float32)

In [22]:
from keras import backend as K
func = K.function(models[0].inputs, [shared_layers[0].output])

AttributeError: Layer conv2d_1 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [21]:
shared_layers[0].call(X)

AttributeError: 'numpy.dtype' object has no attribute 'base_dtype'

In [28]:
(head.predict(X) != 0).sum()

0

In [17]:
import numpy as np

In [26]:
np.stack([np.array([1,2]) for _ in range(2)], axis=1).max(axis=1)

array([1, 2])